In [1]:
import os


In [2]:
os.chdir('../')
os.getcwd()

'/home/who/Documents/Coding/MLP-2_Chicken-Disease-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,config_path = CONFIG_FILE_PATH,params_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ings_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ings_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ings_config

In [6]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size 

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_url,filename = self.config.local_data_file)
            logger.info(f"{filename} download with info:\n{headers}")
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ings_config = config.get_data_ings_config()
    data_ings = DataIngestion(config=data_ings_config)
    data_ings.download_file()
    data_ings.extract_zip_file()
except Exception as e:
    raise e

[2025-01-03 12:14:05,320: INFO: common: YAML file: config/config.yaml read successfully]
[2025-01-03 12:14:05,323: INFO: common: YAML file: params.yaml read successfully]
[2025-01-03 12:14:05,324: INFO: common: Created directory at: 'artifacts']
[2025-01-03 12:14:05,325: INFO: common: Created directory at: 'artifacts/data_ingestion']
[2025-01-03 12:14:11,057: INFO: 2901894440: artifacts/data_ingestion/data.zip download with info:
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8514:15B3EF:8D275D:D23D12:67778733
Accept-Ranges: bytes
Date: Fri, 03 Jan 2025 06:44:06 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100027-HYD
X-Cache: MISS
X-Cac